<a href="https://colab.research.google.com/github/rus4787/Business-evaluation-of-the-method-of-sale/blob/main/%D0%A6%D0%B8%D0%BA%D0%BB_%D0%A1%D0%B4%D0%B5%D0%BB%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных

In [ ]:
import time
import pandas as pd
import numpy as np
import re
import seaborn as sns
import joblib

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)        # Игнорировать FutureWarning
warnings.simplefilter(action='ignore', category=DeprecationWarning)   # Игнорировать DeprecationWarning
warnings.simplefilter(action='ignore', category=SyntaxWarning)        # Игнорировать SyntaxWarning
warnings.simplefilter(action='ignore', category=RuntimeWarning)       # Игнорировать RuntimeWarning
warnings.simplefilter(action='ignore', category=UserWarning)          # Игнорировать UserWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
final_df_2 = joblib.load('/content/drive/MyDrive/barnaul_analitic/base/final_df_2.joblib')
final_df_2['дата_оплаты_счета'] = final_df_2['дата_оплаты_счета'].fillna(pd.Timestamp('2262-04-11'))
completed_deals_df = final_df_2[final_df_2['статус'] == 'завершена']

# 1. График "Цикл сделки по годам"



In [ ]:
import pandas as pd
import plotly.express as px

# Шаг 1: Фильтрация данных для завершенных сделок
completed_deals_df = final_df_2[final_df_2['статус'] == 'завершена']

# Шаг 2: Группировка и расчет среднего цикла сделки по годам
yearly_cycle = completed_deals_df.groupby('год')['цикл сделки'].mean().reset_index()

# Шаг 3: Создание интерактивного графика
fig_1 = px.bar(yearly_cycle, x='год', y='цикл сделки', title='Средний Цикл Сделки по Годам',
             labels={'цикл сделки': 'Средний Цикл Сделки (дней)', 'год': 'Год'})

# Добавляем интерактивность
fig_1.update_layout(bargap=0.2, xaxis_tickformat='%Y', hovermode="x unified")

# Отображение графика 1


In [ ]:
fig_1.show()

# 2. График "Цикл сделки по месяцам в 2024 году"

тренды в цикле сделки по каждому месяцу 2024 года

In [ ]:
# Шаг 1: Фильтрация данных для завершенных сделок в 2024 году и переименование столбца
completed_deals_2024 = completed_deals_df[completed_deals_df['год'] == 2024].copy()
completed_deals_2024 = completed_deals_2024.rename(columns={'месяц': 'месяц_оплаты'})

# Добавление нового столбца "месяц_звонка" на основе даты последнего звонка
completed_deals_2024['месяц_звонка'] = pd.to_datetime(completed_deals_2024['дата_последнего_звонка'], errors='coerce').dt.month
# Шаг 2: Группировка и расчет среднего цикла сделки по месяцам оплаты и месяцам звонков
monthly_cycle_payment = completed_deals_2024.groupby('месяц_оплаты')['цикл сделки'].mean().reset_index()
monthly_cycle_call = completed_deals_2024.groupby('месяц_звонка')['цикл сделки'].mean().reset_index()

# Шаг 3: Создание интерактивного графика с двумя линиями
fig_2 = px.line(monthly_cycle_payment, x='месяц_оплаты', y='цикл сделки',
              title='Средний Цикл Сделки по Месяцам в 2024 году',
              labels={'цикл сделки': 'Средний Цикл Сделки (дней)', 'месяц_оплаты': 'Месяц'},
              markers=True)

# Добавление первой линии с названием в легенде
fig_2.update_traces(name='Средний цикл сделки по месяцу оплаты')

# Добавление второй линии для месяца звонков с названием
fig_2.add_scatter(x=monthly_cycle_call['месяц_звонка'], y=monthly_cycle_call['цикл сделки'],
                mode='lines+markers', name='Средний цикл сделки по месяцу звонка')

# Настройка интерактивности и меток
fig_2.update_layout(xaxis_tickmode="array", xaxis_tickvals=list(range(1, 13)), xaxis_ticktext=[
    "Янв", "Фев", "Март", "Апр", "Май", "Июнь", "Июль", "Авг", "Сент", "Окт", "Ноя", "Дек"
])
fig_2.update_layout(hovermode="x unified")

# Отображение графика 2

In [ ]:
fig_2.show()

# 3. График "Цикл сделки по топ-20 менеджерам по продажам за последний месяц"

In [ ]:
import plotly.express as px

def plot_cycle_by_top_managers(month):
    # Шаг 1: Фильтрация данных для завершенных сделок в 2024 году за указанный месяц
    completed_deals_2024 = completed_deals_df[completed_deals_df['год'] == 2024].copy()
    completed_deals_2024['месяц'] = pd.to_datetime(completed_deals_2024['дата_первого_звонка']).dt.month
    filtered_df = completed_deals_df[completed_deals_df['месяц'] == month]

    # Шаг 2: Группировка и расчет среднего цикла сделки, суммы продаж и количества продаж
    managers_sales = filtered_df.groupby('фио_менеджера').agg({
        'сумма': 'sum',
        'цикл сделки': 'mean',
        'инн_организации': 'count'  # Подсчет количества сделок (продаж) у каждого менеджера
    }).reset_index().rename(columns={'инн_организации': 'количество_продаж'})

    # Сортировка по продажам и выбор топ-20
    top_managers = managers_sales.nlargest(20, 'сумма')

    # Шаг 3: Создание интерактивного графика с дополнительной информацией в hover
    fig = px.bar(top_managers, x='цикл сделки', y='фио_менеджера', orientation='h',
                 title=f'Средний Цикл Сделки по ТОП-20 Менеджерам за {month}-й месяц 2024 года',
                 labels={'фио_менеджера': 'Менеджер', 'цикл сделки': 'Средний Цикл Сделки (дней)'},
                 hover_data={'сумма': True, 'количество_продаж': True})

    # Настройка интерактивности и сортировки
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, hovermode="y unified")

    # Отображение графика
    fig.show()

# Отображение графика 3

In [ ]:
# Вызов функции (для октября = 10)
plot_cycle_by_top_managers(10)

# 4. График "Цикл сделки по классам ОКВЭД за последние три месяца"

топ-10 классов с наибольшим и наименьшим количеством продаж в месяце


In [ ]:
import plotly.express as px
import pandas as pd

def plot_cycle_by_okved(last_month):
    # Шаг 1: Определение последних трех месяцев
    months = [(last_month - i) % 12 or 12 for i in range(3)][::-1]  # Последние три месяца
    year_shift = [(last_month - i - 1) // 12 for i in range(3)]  # Корректировка года для прошлых месяцев

    # Шаг 2: Фильтрация данных за 2024 год и последние три месяца
    completed_deals_2024 = completed_deals_df[completed_deals_df['год'] == 2024].copy()
    completed_deals_2024['месяц'] = pd.to_datetime(completed_deals_2024['дата_оплаты_счета'], errors='coerce').dt.month
    filtered_df = completed_deals_2024[completed_deals_2024['месяц'].isin(months)]

# Шаг 3: Группировка данных и расчет метрик
    okved_sales = filtered_df.groupby(['класс_оквэд', 'месяц']).agg({
        'сумма': 'sum',
        'цикл сделки': 'mean',
        'инн_организации': 'count'  # Количество продаж по каждому ОКВЭД
    }).reset_index().rename(columns={'инн_организации': 'количество_продаж'})

    # Выбор топ-10 и низ-10 классов ОКВЭД по количеству продаж
    top_10_okveds = okved_sales.nlargest(10, 'количество_продаж')
    bottom_10_okveds = okved_sales.nsmallest(10, 'количество_продаж')

    # Шаг 4: Построение стековой диаграммы для топ-10 классов ОКВЭД
    fig1 = px.bar(top_10_okveds, x='месяц', y='цикл сделки', color='класс_оквэд',
                  title='Средний Цикл Сделки по ТОП-10 Классам ОКВЭД за Последние Три Месяца',
                  labels={'цикл сделки': 'Средний Цикл Сделки (дней)', 'месяц': 'Месяц'},
                  hover_data={'сумма': True, 'количество_продаж': True})
    fig1.update_layout(barmode='stack', xaxis_tickmode="array", xaxis_tickvals=months,
                       xaxis_ticktext=[f'{month}-2024' for month in months])
    fig1.update_layout(hovermode="x unified")

    # Шаг 5: Построение стековой диаграммы для низ-10 классов ОКВЭД
    fig2 = px.bar(bottom_10_okveds, x='месяц', y='цикл сделки', color='класс_оквэд',
                  title='Средний Цикл Сделки по Низ-10 Классам ОКВЭД за Последние Три Месяца',
                  labels={'цикл сделки': 'Средний Цикл Сделки (дней)', 'месяц': 'Месяц'},
                  hover_data={'сумма': True, 'количество_продаж': True})
    fig2.update_layout(barmode='stack', xaxis_tickmode="array", xaxis_tickvals=months,
                       xaxis_ticktext=[f'{month}-2024' for month in months])
    fig2.update_layout(hovermode="x unified")

    # Отображение графиков
    fig1.show()
    fig2.show()

# Отображение графика 4

In [ ]:
# Вызов функции (задаем последний месяц, для октября = 10)
plot_cycle_by_okved(10)

# 5. График распределения "Цикл сделки по месяцам за последние три месяца"

In [ ]:
import plotly.express as px
import pandas as pd

def plot_cycle_distribution(last_month):
    # Шаг 1: Определение последних трех месяцев
    months = [(last_month - i) % 12 or 12 for i in range(3)][::-1]  # Последние три месяца
    year_shift = [(last_month - i - 1) // 12 for i in range(3)]  # Корректировка года для прошлых месяцев

    # Шаг 2: Фильтрация данных за 2024 год и последние три месяца
    completed_deals_2024 = completed_deals_df[completed_deals_df['год'] == 2024].copy()
    completed_deals_2024['месяц_звонка'] = pd.to_datetime(completed_deals_2024['дата_последнего_звонка'], errors='coerce').dt.month
    filtered_df = completed_deals_2024[completed_deals_2024['месяц_звонка'].isin(months)]

    # Шаг 3: Создание графика распределения (KDE) для каждого из последних трех месяцев
    fig = px.histogram(filtered_df, x='цикл сделки', color='месяц_звонка', nbins=30,
                       title='Распределение Цикла Сделки по Последним Трем Месяцам',
                       labels={'цикл сделки': 'Цикл Сделки (дней)', 'месяц_звонка': 'Месяц'},
                       marginal="box",  # Добавление боксового графика для детализации распределения
                       opacity=0.7,  # Прозрачность для наложения графиков
                       histnorm='density')  # Нормируем для плотности

    # Настройка оси и подписей
    fig.update_layout(barmode='overlay', hovermode="x unified")
    fig.update_traces(opacity=0.6)
    fig.update_layout(xaxis_title="Цикл Сделки (дней)", yaxis_title="Плотность")

    # Отображение графика
    fig.show()

# Отображение графика 5

In [ ]:
# Вызов функции (задаем последний месяц, для октября = 10)
plot_cycle_distribution(10)

Пояснение по Графику 5:
- График распределения (KDE): сглаженная гистограмма, которая показывает плотность цикла сделок, помогает понять, где находятся основные "скопления" значений. Например, если плотность выше на значении "30 дней", это значит, что в этом месяце больше сделок длились примерно 30 дней.

- Интерактивная на нижней диаграмме:
    - Цикл сделки (дней) — количество дней, сколько длились сделки.
    - density (Плотность) — относительная частота сделок для определённой продолжительности.


- Интерактивная на верхней диаграмме (если значения двойные: (19, 10), то: первое число==значение, второе число==месяц):
    - min: Минимальное значение для цикл сделки в данном месяце (может быть выведенно один раз если одинаково для всех периодов (у нас min==1))
    - месяц: не указан, просто значение (например 10, 9 или 8)
    - lower fence: Нижняя граница для выбросов (значения ниже этой границы считаются выбросами).
    - q1: Первый квартиль (25-й процентиль), указывает нижнюю границу основных данных — 25% значений цикла сделки находятся ниже этого значения.
    - median: Медианное значение, середина данных, где 50% сделок короче этого времени, а 50% — длиннее.
    - q3: Третий квартиль (75-й процентиль), указывает верхнюю границу основных данных — 75% значений цикла сделки находятся ниже этого значения.
    - upper fence: Верхняя граница для выбросов (значения выше этой границы считаются выбросами).
    - max: Максимальное значение цикла сделки в данном месяце (без учета выбросов).

# выбор метрики для анализа цикла сделки:

- Месяц первого звонка:
    - Что показывает: Анализ сделок по месяцу первого звонка отражает, как быстро удается завершить сделки, начиная с первого контакта. Это полезно, если вы хотите оценить, насколько эффективно менеджеры доводят новые контакты до сделки.
    - Плюсы: Подходит для оценки времени конверсии с момента первого контакта.
    - Минусы: Может искажать данные, если на сделки влияет большое количество последующих звонков или промежуток времени между первым звонком и оплатой сильно варьируется.

- Месяц последнего звонка:
    - Что показывает: Смотрим на месяц, в котором состоялся последний разговор перед оплатой. Этот подход отражает, когда был активен последний этап коммуникации с клиентом.
    - Плюсы: Показывает, как часто требуется взаимодействие перед заключением сделки. Подходит для анализа финальных этапов продаж.
    - Минусы: Если сделка затягивалась, и последний звонок был значительно позже первого, это может создать иллюзию более длинного цикла.

- Месяц оплаты:
    - Что показывает: Ориентируется на дату фактического завершения сделки (когда оплата получена).
    - Плюсы: Счет закрывается по фактическому доходу в месяц оплаты, что полезно для финансового планирования и анализа доходов по месяцам.
    - Минусы: Этот подход может скрывать длинные циклы продаж, если период от первого контакта до оплаты значителен.

- Рекомендации
    - Для анализа эффективности работы менеджеров: месяц первого звонка, поскольку он показывает, как быстро удается конвертировать новый контакт в сделку.
    - Для анализа активности в завершении сделок: месяц последнего звонка, так как он помогает понять, насколько эффективно менеджеры доводят потенциальные сделки до завершения в финальной стадии.
    - Для финансового анализа: месяц оплаты, так как этот подход точно отражает доходность по месяцам.